In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import BinaryAccuracy, AUC
from keras.regularizers import l1_l2

from tensorflow.keras.callbacks import EarlyStopping

2022-10-04 08:38:21.877056: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/Immobilienrechner-Challenge/data/main/immoscout_cleaned_lat_lon_fixed_v9.csv")
data

/shared-libs/python3.8/py-core/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,...,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,price_cleaned,type,Space extracted,rooms
0,0,0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,...,2.234259,5.89,14.0,9.0,308.0,331.0,1150000.0,penthouse,100.0,5.0
1,1,1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,...,2.234259,5.89,14.0,9.0,308.0,331.0,1420000.0,terrace-house,156.0,5.0
2,2,2,NaN,NaN,NaN,NaN,NaN,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",...,3.549010,6.05,37.0,3092.0,30364.0,33493.0,720000.0,penthouse,93.0,5.0
3,3,3,Biberstein,154 m²,370 m²,257 m²,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,...,2.234259,5.89,14.0,9.0,308.0,331.0,1430000.0,detached-house,154.0,5.0
4,4,4,Küttigen,142 m²,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,...,1.708126,6.30,65.0,349.0,941.0,1355.0,995000.0,flat,142.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13373,13373,13373,Uster,70 m²,NaN,NaN,On request,"Tägerackerstrasse 138610 Uster, ZH","2.5 rooms, 70 m²«Wunderschöne Attikawohnung an...",DescriptionIn einem der schönsten Quartiere Us...,...,1.567019,4.52,181.0,3218.0,12931.0,16330.0,1101000.0,flat,70.0,5.0
13374,13374,13374,Volketswil,NaN,284 m²,NaN,On request,"8604 Volketswil, ZH",8 rooms«Charmantes 2-Familienhaus im idyllisch...,DescriptionWillkommen im Herzen von Volketswil...,...,2.368364,4.23,74.0,3406.0,7644.0,11124.0,1750000.0,semi-detached-house,NaN,8.0
13375,13375,13375,Seuzach,150 m²,160 m²,NaN,On request,"8472 Seuzach, ZH","6.5 rooms, 150 m²«EIN PARADIES IM GRÜNEN FÜR G...",DescriptionWir verkaufen dieses unglaublich to...,...,1.991008,4.33,56.0,212.0,358.0,626.0,1415000.0,terrace-house,150.0,5.0
13376,13376,13376,Zell (ZH),145 m²,853 m²,140 m²,Immediately,"Vordere Bähntalstrasse 18483 Kollbrunn, ZH","6 rooms, 145 m²«Freistehendes 6.0 Zimmer . Ein...",Description####Freistehendes 6.0 Zimmer . Einf...,...,1.257283,4.35,66.0,167.0,624.0,857.0,1465000.0,detached-house,145.0,6.0


In [3]:
data["type"].unique()

array(['penthouse', 'terrace-house', 'detached-house', 'flat',
       'stepped-house', 'farmhouse', 'semi-detached-house',
       'stepped-apartment', 'duplex-maisonette', 'attic-flat', 'loft',
       'chalet', 'villa', 'attic-room', 'secondary-suite', 'castle',
       'detached-secondary-suite', 'studio',
       'furnished-residential-property', 'rustico', 'single-room'],
      dtype=object)

In [4]:
data = pd.get_dummies(data=data, columns=["type"])
data

,Unnamed: 0,Unnamed: 0.1,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,...,type_penthouse,type_rustico,type_secondary-suite,type_semi-detached-house,type_single-room,type_stepped-apartment,type_stepped-house,type_studio,type_terrace-house,type_villa
0,0,0,Biberstein,100 m²,NaN,NaN,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,...,1,0,0,0,0,0,0,0,0,0
1,1,1,Biberstein,156 m²,222 m²,242 m²,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,...,0,0,0,0,0,0,0,0,1,0
2,2,2,NaN,NaN,NaN,NaN,NaN,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",...,1,0,0,0,0,0,0,0,0,0
3,3,3,Biberstein,154 m²,370 m²,257 m²,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,...,0,0,0,0,0,0,0,0,0,0
4,4,4,Küttigen,142 m²,NaN,NaN,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13373,13373,13373,Uster,70 m²,NaN,NaN,On request,"Tägerackerstrasse 138610 Uster, ZH","2.5 rooms, 70 m²«Wunderschöne Attikawohnung an...",DescriptionIn einem der schönsten Quartiere Us...,...,0,0,0,0,0,0,0,0,0,0
13374,13374,13374,Volketswil,NaN,284 m²,NaN,On request,"8604 Volketswil, ZH",8 rooms«Charmantes 2-Familienhaus im idyllisch...,DescriptionWillkommen im Herzen von Volketswil...,...,0,0,0,1,0,0,0,0,0,0
13375,13375,13375,Seuzach,150 m²,160 m²,NaN,On request,"8472 Seuzach, ZH","6.5 rooms, 150 m²«EIN PARADIES IM GRÜNEN FÜR G...",DescriptionWir verkaufen dieses unglaublich to...,...,0,0,0,0,0,0,0,0,1,0
13376,13376,13376,Zell (ZH),145 m²,853 m²,140 m²,Immediately,"Vordere Bähntalstrasse 18483 Kollbrunn, ZH","6 rooms, 145 m²«Freistehendes 6.0 Zimmer . Ein...",Description####Freistehendes 6.0 Zimmer . Einf...,...,0,0,0,0,0,0,0,0,0,0


In [5]:
unnamed_cols = data.columns[data.columns.str.startswith("Unnamed: ")]
y_cols = data.columns[data.columns.str.startswith("type_")]
X_cols = list(set(data._get_numeric_data().columns) - set(y_cols) - set(unnamed_cols))

X = data[X_cols]
y = data[y_cols]

In [6]:
X = normalize(X, axis=-1, order=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
n_epochs = 200
batchsize = 2 ** 8

n_inputs = X_train.shape[1]
n_outputs = y_train.shape[1]

n_hiddenlayers = 2
n_nodes_per_layer = 128


model = Sequential()

for i in range(n_hiddenlayers):
    model.add(
        Dense(n_nodes_per_layer, activation='relu', input_shape=(n_inputs,))
    )

model.add(
        Dense(n_outputs, activation='sigmoid')
)

model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy', 'AUC']
)

early_stop = EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        verbose=1, 
        patience=25
)

history = model.fit(
        X_train, 
        y_train, 
        epochs=n_epochs, 
        batch_size=batchsize, 
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        use_multiprocessing=True
)

Epoch 1/200
2022-10-04 08:38:24.434838: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (34)
2022-10-04 08:38:24.434872: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-985da17b-3a43-4cd2-aed4-b3aeabe863c4): /proc/driver/nvidia/version does not exist
2022-10-04 08:38:24.435099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
42/42 [==============================] - 1s 11ms/step - loss: 0.6652 - accuracy: 0.4382 - auc: 0.5654 - val_loss: 0.6205 - val_accuracy: 0.4477 - val_auc: 0.6663
Epoch 2/200
42/42 [==============================] - 0s 6ms/step - loss: 0.5478 - accuracy: 0.4482 - auc: 0.650

In [8]:
for i, val in enumerate(model.predict(X_test.head(1))[0]):
    print(f'{y.columns[i]}: {val:.2f}')

1/1 [==============================] - 0s 64ms/step
type_attic-flat: 0.02
type_attic-room: 0.00
type_castle: 0.00
type_chalet: 0.02
type_detached-house: 0.27
type_detached-secondary-suite: 0.00
type_duplex-maisonette: 0.03
type_farmhouse: 0.01
type_flat: 0.44
type_furnished-residential-property: 0.00
type_loft: 0.00
type_penthouse: 0.04
type_rustico: 0.00
type_secondary-suite: 0.00
type_semi-detached-house: 0.04
type_single-room: 0.00
type_stepped-apartment: 0.01
type_stepped-house: 0.00
type_studio: 0.00
type_terrace-house: 0.04
type_villa: 0.05


In [9]:
for i, val in enumerate(y_test.head(1).values[0]):
    print(f'{y.columns[i]}: {val:.0f}')

type_attic-flat: 0
type_attic-room: 0
type_castle: 0
type_chalet: 0
type_detached-house: 0
type_detached-secondary-suite: 0
type_duplex-maisonette: 0
type_farmhouse: 0
type_flat: 1
type_furnished-residential-property: 0
type_loft: 0
type_penthouse: 0
type_rustico: 0
type_secondary-suite: 0
type_semi-detached-house: 0
type_single-room: 0
type_stepped-apartment: 0
type_stepped-house: 0
type_studio: 0
type_terrace-house: 0
type_villa: 0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=985da17b-3a43-4cd2-aed4-b3aeabe863c4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>